In [ ]:
!pip install tensorflow==2.12.0
!pip install pandas
!pip install numpy

import os
import torch
import torchvision
from torch import nn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [ ]:
train_datasets = "archive/Training"
test_datasets = "archive/Testing"

----

In [ ]:
# PyTorch transform image set up

transform_train = torchvision.transforms.Compose([
    # Resize all images to 224 x 224 pixel image
    torchvision.transforms.Resize(230,230),

    # Give the image a random crop
    torchvision.transforms.RandomRotation(10),
    torchvision.transforms.RandomResizedCrop(size=(224,224), scale=(0.8,1.0), ratio=(1.0,1.0)),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomVerticalFlip(),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],std=[0.2023, 0.1994, 0.2010])
])


transform_test = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],std=[0.2023, 0.1994, 0.2010])
])

In [ ]:
# Arrays to hold all images and labels

train_data = []
train_labels = []

test_data = []
test_labels = []

In [ ]:
# Import images
train_data = [torch.utils.data.DataLoader()]